# OpenCV中的绘图功能
**cv.line()，cv.circle()，cv.rectangle()，cv.ellipse()，cv.putText()**

在上述所有功能中，您将看到一些常见的参数，如下所示：
* img：您要绘制形状的图像
* color：形状的颜色。对于BGR，将其作为元组传递，例如：(255,0,0)对于蓝色。对于灰度，只需传递标量值即可。
* 厚度：线或圆等的粗细。如果对闭合图形（如圆）传递-1 ，它将填充形状。默认厚度= 1
* lineType：线的类型，是否为8连接线，抗锯齿线等。默认情况下，为8连接线。**cv.LINE_AA**给出了抗锯齿的线条，看起来非常适合曲线。

In [1]:
import numpy as np
import cv2 as cv

## 画线
开始，结束坐标

In [3]:
img = np.zeros((512, 512, 3), np.uint8) # 创建黑色图象
cv.line(img, (0,0), (511,511), (255,0,0), 5)
cv.imshow('image', img)
cv.waitKey(0)
cv.destroyAllWindows()

## 画矩形
左上角， 右下角

横轴x 竖轴y

In [6]:
cv.rectangle(img, (384,0), (510,128), (0, 255, 0), 3)
cv.imshow('image', img)
cv.waitKey(0)
cv.destroyAllWindows()

## 画圆
圆心，半径

In [9]:
cv.circle(img, (447,63), 63, (0,0,255), 5)
cv.imshow('image', img)
cv.waitKey(0)
cv.destroyAllWindows()

[![5piBB8.png](https://z3.ax1x.com/2021/10/07/5piBB8.png)](https://imgtu.com/i/5piBB8)

## 画椭圆
* 中心(x,y)，
* (长轴，短轴)
* angle是椭圆沿逆时针方向旋转的角度。
* startAngle和endAngle表示从主轴沿顺时针方向测量的椭圆弧的开始和结束。即给出0和360给出完整的椭圆

In [10]:
cv.ellipse(img, (256,256), (100, 50), 0, 0, 180, 255, -1)
cv.imshow('image', img)
cv.waitKey(0)
cv.destroyAllWindows()

[![5pFu5Q.png](https://z3.ax1x.com/2021/10/07/5pFu5Q.png)](https://imgtu.com/i/5pFu5Q)